In [1]:
import numpy as np
import json 
import pandas as pd


pathToCsvFile = "../../part1/athlete_events.csv"
gamesCollectionFile = "games.json" 
athletesCollectionFile = "athletes.json" 
sportsCollectionFile = "sports.json" 

# Data exploration

In [2]:
df = pd.read_csv(pathToCsvFile, header=0)
df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


# Printing the unique value count for each column

In [3]:
print("Event",len(df.Event.unique()))

print("Name",len(df.Name.unique()))

print("Team",len(df.Team.unique()))

print("Sport",len(df.Sport.unique()))

print("Medal",len(df.Medal.unique()))

print("Games",len(df.Games.unique()))

Event 765
Name 134732
Team 1184
Sport 66
Medal 4
Games 51


In [4]:
def getEntityToIdDict(entityArray):
    entityToId = {}
    i = 1
    for i in range(len(entityArray)):
        entity = entityArray[i]
        if entity not in entityToId:
            entityToId[entity] = i + 1
            i += 1
    return entityToId
    
eventToId = getEntityToIdDict(df.Event.unique())
athleteToId = getEntityToIdDict(df.Name.unique())
teamToID = getEntityToIdDict(df.Team.unique())
sportToID = getEntityToIdDict(df.Sport.unique())
medalToID = getEntityToIdDict(df.Medal.unique())
gamesToID = getEntityToIdDict(df.Games.unique())
        
        
def writeToFile(filename, entries):
    print("Filename", filename, "Total entries: ", len(entries))
    with open(filename, 'w') as f:
        json.dump(entries, f, indent=4)

# Games Collection

In [164]:
gameTemplate = """
{{"gameId" : {gameId}, "season" : {season}, "year" :  {year}, "city" : {city}, "teams": {teams}}}
"""

gameIdToGame = {} # gameId : games
gameIdToTeamIds = {} # gameId : set
gameIdToTeams = {} # gameId : [team]

games = []

for ind in df.index:
    gameId = gamesToID[df.Games[ind]]
    season = df.Season[ind]
    year = df.Year[ind]
    city = df.City[ind]
    teamId = teamToID[df.Team[ind]]
    country = df.Team[ind]
    noc = df.NOC[ind]
    
    if gameId not in gameIdToGame:
        game = {
                    "gameId": gameId,
                    "season" : "\"" + season + "\"",
                    "year" :  year,
                    "city" : "\"" + city + "\""
        }
        gameIdToGame[gameId] = game
        
    team = {
        "teamId" : teamId,
        "country" : country, 
        "noc" :  noc
    }
    
    if gameId not in gameIdToTeamIds:
        gameIdToTeamIds[gameId] = set()
        gameIdToTeams[gameId] = []
        
    if not (teamId in gameIdToTeamIds[gameId]):
        gameIdToTeamIds[gameId].add(teamId)
        gameIdToTeams[gameId].append(team)
    

for gameId in gameIdToGame:
    teams = gameIdToTeams[gameId]
    season = gameIdToGame[gameId]["season"]
    year = gameIdToGame[gameId]["year"]
    city = gameIdToGame[gameId]["city"]
    game = gameTemplate.format(gameId=gameId, season=season, year=year, city=city, teams=teams)
    game = eval(game)
    games.append(game)


print(games[0])    
writeToFile(gamesCollectionFile, games)


{'gameId': 1, 'season': 'Summer', 'year': 1992, 'city': 'Barcelona', 'teams': [{'teamId': 1, 'country': 'China', 'noc': 'CHN'}, {'teamId': 6, 'country': 'Finland', 'noc': 'FIN'}, {'teamId': 7, 'country': 'Norway', 'noc': 'NOR'}, {'teamId': 9, 'country': 'Estonia', 'noc': 'EST'}, {'teamId': 18, 'country': 'Chad', 'noc': 'CHA'}, {'teamId': 23, 'country': 'Cuba', 'noc': 'CUB'}, {'teamId': 28, 'country': 'Chile', 'noc': 'CHI'}, {'teamId': 32, 'country': 'Hungary', 'noc': 'HUN'}, {'teamId': 17, 'country': 'Italy', 'noc': 'ITA'}, {'teamId': 37, 'country': 'Pakistan', 'noc': 'PAK'}, {'teamId': 14, 'country': 'Egypt', 'noc': 'EGY'}, {'teamId': 34, 'country': 'Algeria', 'noc': 'ALG'}, {'teamId': 41, 'country': 'Qatar', 'noc': 'QAT'}, {'teamId': 55, 'country': 'Djibouti', 'noc': 'DJI'}, {'teamId': 42, 'country': 'Malaysia', 'noc': 'MAS'}, {'teamId': 38, 'country': 'Iraq', 'noc': 'IRQ'}, {'teamId': 35, 'country': 'Kuwait', 'noc': 'KUW'}, {'teamId': 69, 'country': 'Singapore', 'noc': 'SGP'}, {'tea

# Athletes Collection

In [165]:
athletesTemplate = """
{{"athleteId" : {athleteId}, "name" : {name}, "sex": {sex}, "participations" : {participations}}}
"""

athleteIdToAthletes = {}
athleteIdToKey = {}
athleteIdParticipation = {}

athletes = []
for ind in df.index:
    athleteId = athleteToId[df.Name[ind]]
    name = df.Name[ind]
    sex = df.Sex[ind]
    sportId = sportToID[df.Sport[ind]]
    gameId = gamesToID[df.Games[ind]]
    teamId = teamToID[df.Team[ind]]
    age = df.Age[ind]
    height = df.Height[ind]
    weight = df.Weight[ind]
    medalType = df.Medal[ind]
    if not pd.isna(df.Age[ind]) and  (not pd.isna(df.Name[ind])) and (not pd.isna(df.Sex[ind])) and (not pd.isna(df.Height[ind])) and (not pd.isna(df.Weight[ind])):
        key = str(sportId) + "-" + str(gameId) + "-"  + str(teamId)
        if athleteId not in athleteIdToAthletes:
            athlete = {
                        "athleteId": athleteId,
                        "name" : "\""+ name.replace("\"","") + "\"",
                        "sex" : "\"" + sex + "\""
            }
            athleteIdToAthletes[athleteId] = athlete
            
        if pd.isna(df.Medal[ind]) :
            medalType = "Null"

        participation = {
                    "sportId" : sportId,
                    "gameId" :  gameId,
                    "teamId" :  teamId,
                    "age" : int(age),
                    "height" : height,
                    "weight" :  weight,
                    "medalType" :  medalType
        }
        if athleteId not in athleteIdToKey:
            athleteIdToKey[athleteId] = set()
            athleteIdParticipation[athleteId] = []
        
        # print(key, not (key  in athleteIdToKey[athleteId]), athleteIdToKey[athleteId])
        if not (key in athleteIdToKey[athleteId]):
            athleteIdToKey[athleteId].add(key)
            athleteIdParticipation[athleteId].append(participation)
        
for athleteId in athleteIdToAthletes:
    participations = athleteIdParticipation[athleteId]
    name = athleteIdToAthletes[athleteId]["name"]
    sex = athleteIdToAthletes[athleteId]["sex"]
    athlete = athletesTemplate.format(athleteId=athleteId, name=name,sex=sex, participations=participations)
    athlete = eval(athlete)
    athletes.append(athlete)
print(athletes[0])
writeToFile(athletesCollectionFile, athletes)

{'athleteId': 1, 'name': 'A Dijiang', 'sex': 'M', 'participations': [{'sportId': 1, 'gameId': 1, 'teamId': 1, 'age': 24, 'height': 180.0, 'weight': 80.0, 'medalType': 'Null'}]}
Filename athletes.json Total entries:  98546


# Sports Collection

In [6]:
sportsTemplate = """
{{"sportId" : {sportId}, "sportsType" : {sportsType}, "events": {events}}}
"""

sportIdToSports = {} # sportId : Sports
sportIdToEventId = {} # sportId : set
sportIdEvents = {} # sportId : [events]



sports = []
for ind in df.index:
    sportId = sportToID[df.Sport[ind]]
    sportsType = df.Sport[ind]
    eventId = eventToId[df.Event[ind]]
    eventType = df.Event[ind]
    
    if sportId not in sportIdToSports:
        sport = {
                    "sportId": sportId,
                    "sportsType" : "\"" + sportsType + "\""    
        }
        sportIdToSports[sportId] = sport
    
    event = {"eventId" : eventId, "eventType" : eventType}
        
    if sportId not in sportIdToEventId:
        sportIdToEventId[sportId] = set()
        sportIdEvents[sportId] = []
        
    if not (eventId in sportIdToEventId[sportId]):
        sportIdToEventId[sportId].add(eventId)
        sportIdEvents[sportId].append(event)
        
        
for sportId in sportIdToSports:
    events = sportIdEvents[sportId]
    sportsType = sportIdToSports[sportId]["sportsType"]
    sport = sportsTemplate.format(sportId=sportId, sportsType=sportsType,events=events)
    sport = eval(sport)
    sports.append(sport)

print(sports[0]) 
writeToFile(sportsCollectionFile, sports)

{'sportId': 1, 'sportsType': 'Basketball', 'events': [{'eventId': 1, 'eventType': "Basketball Men's Basketball"}, {'eventId': 83, 'eventType': "Basketball Women's Basketball"}]}
Filename sports.json Total entries:  66
